$\textbf{Exercise 2}$

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import random
from sklearn.datasets import load_digits
import timeit
random.seed(10)

In [6]:
def evalf(A,x,y,n_feat, n, lamb):


  f = np.matmul(A,x) - y

  return 0.5*(np.linalg.norm(f))**2  + 0.5*lamb*(np.matmul(x.T , x))


def evalg(A,x,y,n_feat, n, lamb):


  d = np.matmul(A,x) - y
  g = np.matmul(A.T , d) + lamb*x

  return g



def evalh(A,x,y,n_feat, n, lamb):


  h = np.matmul(A.T , A) + lamb*np.identity(n_feat)

  return h


In [7]:
def compute_steplength_backtracking_scaled_direction(A,x,y, n_feat, n ,lamb  ,gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function


  alpha = alpha_start
  p = -gradf

  while (evalf(A, x + alpha*np.matmul(direction,p), y, n_feat, n,lamb) > (evalf(A,x,y,n_feat,n,lamb) + gamma * alpha * np.matmul(gradf.T, np.matmul(direction, p))) ):
    alpha = alpha*rho

  return alpha


#line search type
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1



def find_minimizer_Newtonmethod(A, start_x, y, n_feat,  n,lamb, tol, line_search_type, *args):


  x = start_x
  g_x = evalg(A,x,y,n_feat,n,lamb)


  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  x_k = []



  while (np.linalg.norm(g_x) > tol):

    d = np.linalg.inv(evalh(A,x,y,n_feat,n,lamb))

    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(A,x,y,n_feat,n,lamb,g_x, d , alpha_start,rho, gamma)

    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0

    else:
      raise ValueError('Line search type unknown. Please check!')

    x = np.subtract(x, step_length * np.matmul(d,g_x))
    k += 1
    x_k.append(x)
    g_x = evalg(A,x,y,n_feat,n,lamb)

  return x, k, x_k



def find_minimizer_BFGS(A, start_x, y, n_feat, n,lamb, tol,line_search_type,*args):



  x = start_x
  k = 0
  g_new = evalg(A,x,y,n_feat,n,lamb)
  B = np.identity(n_feat)
  x_k = []

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  while (np.linalg.norm(g_new) > tol):

    d = B

    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(A,x,y,n_feat,n,lamb ,g_new, d , alpha_start,rho, gamma)

    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0

    else:
      raise ValueError('Line search type unknown. Please check!')

    g_old = evalg(A,x,y,n_feat,n,lamb)
    p = np.matmul(-1*B, g_old)
    x = x + step_length * p
    s = step_length * p
    g_new = evalg(A,x,y,n_feat,n,lamb)
    y_k = g_new - g_old

    mu = 1 / (np.matmul(y_k.T,s))
    term1 = np.identity(n_feat) - mu * np.matmul(s,y_k.T)
    term2 = np.identity(n_feat) - mu * np.matmul(y_k, s.T)

    B = np.matmul(term1, np.matmul(B ,term2))  +  mu * np.matmul(s,s.T)

    k = k + 1
    x_k.append(x)


  return x, k, x_k

In [9]:
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 1000000]
lambda_reg = 0.1
eps = np.random.randn(N,1)

newton_times = []
bfgs_times = []
newton_val_one = []
newton_val_two = []
bfgs_val_one = []
bfgs_val_two = []

for i in range(len(ds)):

  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()

  x_opt, k_opt , xk = find_minimizer_Newtonmethod(A,xorig,y,d,N,lambda_reg,10e-5,BACKTRACKING_LINE_SEARCH, 0.9, 0.5,0.5)

  newtontime = timeit.default_timer() - start
  newton_times.append(newtontime)
  print('Time for d = {} by Newtons method  :'.format(ds[i]),newtontime)
  print('||Ax_opt-y||^2: ', (np.linalg.norm(np.matmul(A,x_opt) - y))**2  )
  print('||x_opt-x_org||: ', (np.linalg.norm(x_opt - xorig)**2  ))

  newton_val_one.append((np.linalg.norm(np.matmul(A,x_opt) - y))**2 )
  newton_val_two.append((np.linalg.norm(x_opt - xorig)**2 ))



  start = timeit.default_timer()
  x_opt, k_opt , xk = find_minimizer_BFGS(A,xorig,y,d,N,lambda_reg,10e-5,BACKTRACKING_LINE_SEARCH, 0.9, 0.5,0.5)
  bfgstime =  timeit.default_timer() - start
  bfgs_times.append(bfgstime)
  print('Time for d = {} by BFGS method  :'.format(ds[i]),newtontime)
  print('||Ax_opt-y||^2: ', (np.linalg.norm(np.matmul(A,x_opt) - y))**2  )
  print('||x_opt-x_org||: ', (np.linalg.norm(x_opt - xorig)**2  ) )
  bfgs_val_one.append( (np.linalg.norm(np.matmul(A,x_opt) - y))**2)
  bfgs_val_two.append((np.linalg.norm(x_opt - xorig)**2))


Time for d = 1000 by Newtons method  : 1.6451185969999642
||Ax_opt-y||^2:  0.0025220977361562016
||x_opt-x_org||:  793.0785506665048
Time for d = 1000 by BFGS method  : 1.6451185969999642
||Ax_opt-y||^2:  0.0025220987468447744
||x_opt-x_org||:  793.0787084511468
Time for d = 5000 by Newtons method  : 108.64622858800021
||Ax_opt-y||^2:  0.00040041867466264795
||x_opt-x_org||:  4810.7981627740355


KeyboardInterrupt: 

1. The total CPU time taken to solve the respective method

2. The value $ \|\mathbf{Ax}^* - \mathbf{y}\|^2_2$ , where $x^*$ is the respective optimizer obtained by Newton and BFGS methods.

3. The difference of values $ \|\mathbf{x}^* -\mathbf{x_{orig}}\|^2_2$ , where $x^*$ is the respective optimizer obtained by Newton and BFGS
methods and $x_{orig}$ is used in the code.



In [16]:
import pandas as pd

dic= {'Time_newton' : newton_times , 'Time_BFGS' : bfgs_times, '||Ax*-y||_newton' : newton_val_one ,
      '||x*-x_orig||_newton' : newton_val_two,  '||Ax*-y||_bfgs' : bfgs_val_one , '||x*-x_orig||_bfgs' : bfgs_val_two  }

df = pd.DataFrame(dic, index=[1000 , 5000])
df

,Time_newton,Time_BFGS,||Ax*-y||_newton,||x*-x_orig||_newton,||Ax*-y||_bfgs,||x*-x_orig||_bfgs
1000,1.645119,35.347136,0.002522,793.078551,0.002522,793.078708
5000,108.646229,35.347136,0.000400,4810.798163,0.002522,793.078708


In [ ]:
#  The  newton's methods does not show result after d=10000 and for BFGS methods  after d=5000